## **IMPORTACIONES**

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

Bibliotecas importadas.


## **API Key**

In [2]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if api_key:
    print("API Key cargada.")
else:
    print("No se encontró la API Key.")

API Key cargada.


### **Carga de PDFs**
- Cargamos los PDFs, separamos las páginas y le damos una fuente a cada una según al PDF que pertenece.

In [8]:
def load_and_tag_pdfs(data_dir: str, paper_sources: dict):
   
    documents = []

    for filename, source_title in paper_sources.items(): # Itera en el diccionario por cada PDF (toma el nombre del archivo y el título de la fuente) 
        file_path = os.path.join(data_dir, filename)
        
        if not os.path.exists(file_path):
            print(f"ADVERTENCIA: No se encontró el archivo {file_path}")
            continue
            
        try:
            loader = PyPDFLoader(file_path) # Carga el PDF
            pages = loader.load() # Carga las páginas del PDF
            
            for page in pages: # Etiqueta cada página con el título de la fuente (cada página de BERT.pdf tendrá la metadata "source": "BERT")
                page.metadata["source"] = source_title
                
            documents.extend(pages) # Agrega las páginas etiquetadas a la lista de documentos
            print(f"Cargado y etiquetado: {filename} ({len(pages)} páginas)")
            
        except Exception as e:
            print(f"Error al cargar {file_path}: {e}")
            
    return documents

### **Directorio y diccionario de los PDFs**
- Creamos los parametros que va a necesitar la función que creamos previamente.

In [9]:
# Directorio de los PDFs
DATA_FOLDER = "./docs" 

# Cada PDF y su título real (los títulos servirán para etiquetar las páginas)
PAPERS = {
    "AIAYN.pdf": "Attention Is All You Need (Vaswani et al., 2017)",
    "BERT.pdf": "BERT: Pre-training of Deep Bidirectional Transformers (Devlin et al., 2018)",
    "RAG.pdf": "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks (Lewis et al., 2020)"
}

# Llamamos a la funcion que creamos previamente pasandole el directorio y los PDFs
all_documents = load_and_tag_pdfs(DATA_FOLDER, PAPERS)

if all_documents:
    print(f"\nTotal de páginas cargadas: {len(all_documents)}")
    print(f"Fuente de la página 1: {all_documents[0].metadata['source']}")

Cargado y etiquetado: AIAYN.pdf (15 páginas)
Cargado y etiquetado: BERT.pdf (16 páginas)
Cargado y etiquetado: RAG.pdf (19 páginas)

Total de páginas cargadas: 50
Fuente de la página 1: Attention Is All You Need (Vaswani et al., 2017)
